In [1]:
# r"C:\Users\ungdu\Downloads\Chat_Mini\mini_data.csv"

In [22]:
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import chromadb
import uuid
import time

# Hàm xóa ký tự đặc biệt
def remove_special_characters(text):
    text = re.sub(r'<.*?>', ' ', text)  # Loại bỏ HTML tags
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', text)  # Loại bỏ URL
    text = re.sub(r'[^\w\s.!?@]', ' ', text)  # Loại bỏ ký tự đặc biệt
    return text

# Hàm chuyển chữ về chữ thường
def lowercase(text):
    return text.lower()

# Hàm loại bỏ khoảng trắng thừa
def remove_extra_whitespaces(text):
    text = text.strip()  # Xóa khoảng trắng đầu và cuối
    text = re.sub(r'\s+', ' ', text)  # Xóa khoảng trắng thừa trong chuỗi
    return text

# Hàm tổng hợp để tiền xử lý văn bản
def preprocess_text(text):
    text = lowercase(text)
    text = remove_special_characters(text)
    text = remove_extra_whitespaces(text)
    return text

# Hàm xóa các dòng trùng lặp dựa trên một cột cụ thể
def remove_duplicate_rows(df, column_name):
    df.drop_duplicates(subset=column_name, keep='first', inplace=True)
    return df

# Hàm chunking semantic để chia đoạn văn
class SemanticChunker:
    def __init__(self, threshold=0.3):
        self.threshold = threshold
        nltk.download("punkt", quiet=True)

    def embed_function(self, sentences):
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform(sentences).toarray()

        # Lưu TF-IDF vào file CSV
        tfidf_df = pd.DataFrame(vectors, index=sentences, columns=vectorizer.get_feature_names_out())
        tfidf_df.to_csv("tfidf_values.csv", index=True)

        return vectors

    def split_text(self, text):
        sentences = nltk.sent_tokenize(text)  # Tách câu
        sentences = [item for item in sentences if item and item.strip()]
        if not sentences:
            return []

        vectors = self.embed_function(sentences)
        similarities = cosine_similarity(vectors)

        # Lưu cosine similarity vào file CSV
        cosine_df = pd.DataFrame(similarities, index=sentences, columns=sentences)
        cosine_df.to_csv("cosine_similarity.csv", index=True)

        chunks = [[sentences[0]]]  # Bắt đầu chunk đầu tiên
        for i in range(1, len(sentences)):
            sim_score = similarities[i-1, i]
            if sim_score >= self.threshold:
                chunks[-1].append(sentences[i])
            else:
                chunks.append([sentences[i]])

        return [' '.join(chunk) for chunk in chunks]

# Hàm chia DataFrame thành các batch
def divide_dataframe(df, batch_size):
    return [df.iloc[i:i + batch_size] for i in range(0, len(df), batch_size)]

# Đọc file CSV đầu vào
input_file = r"C:\Users\ungdu\Downloads\Chat_Mini\mini_data.csv"
output_file = "processed_data.csv"
chunked_file = "chunked_data.csv"
embedding_file = "embedding_data.csv"

try:
    # Đọc dữ liệu từ file CSV
    df = pd.read_csv(input_file)

    # Kiểm tra nếu DataFrame không rỗng
    if not df.empty:
        # Tiền xử lý cột "Câu hỏi"
        if 'Câu hỏi' in df.columns:
            df['Câu hỏi'] = df['Câu hỏi'].apply(preprocess_text)

        # Xóa các dòng trùng lặp dựa trên cột 'Câu hỏi' nếu tồn tại
        if 'Câu hỏi' in df.columns:
            df = remove_duplicate_rows(df, 'Câu hỏi')

        # Lưu dữ liệu đã tiền xử lý ra file mới
        df.to_csv(output_file, index=False)
        print(f"Dữ liệu đã được xử lý và lưu vào {output_file}")

        # Chunking dữ liệu
        chunker = SemanticChunker(threshold=0.3)
        if 'Câu hỏi' in df.columns:
            df['chunk'] = df['Câu hỏi'].apply(lambda x: chunker.split_text(x))

        # Lưu dữ liệu đã chunking ra file mới
        df.to_csv(chunked_file, index=False)
        print(f"Dữ liệu đã được xử lý và lưu vào {chunked_file}")

        # Embedding dữ liệu
        embedding_model = SentenceTransformer('keepitreal/vietnamese-sbert')
        df['embedding'] = df['chunk'].apply(lambda x: embedding_model.encode(' '.join(x)))

        # Kết nối với Chroma và lưu dữ liệu theo batch
        client = chromadb.PersistentClient("db")
        collection = client.get_or_create_collection("embeddings_collection")
        batch_size = 256
        batches = divide_dataframe(df, batch_size)
        
        for i, batch in enumerate(batches):
            ids = [str(uuid.uuid4()) for _ in range(len(batch))]
            documents = batch['Câu hỏi'].tolist()
            embeddings = batch['embedding'].tolist()
            metadatas = [{"chunk": ' '.join(chk)} for chk in batch['chunk']]

            collection.add(
                ids=ids,
                documents=documents,
                embeddings=embeddings,
                metadatas=metadatas
            )

        # Lưu embedding ra file CSV
        df.to_csv(embedding_file, index=False)
        print(f"Embedding đã được lưu vào {embedding_file}")
    else:
        print("Dữ liệu đầu vào rỗng!")

except FileNotFoundError:
    print(f"Không tìm thấy file {input_file}!")
except Exception as e:
    print(f"Đã xảy ra lỗi: {str(e)}")


Dữ liệu đã được xử lý và lưu vào processed_data.csv
Dữ liệu đã được xử lý và lưu vào chunked_data.csv
Embedding đã được lưu vào embedding_data.csv


In [ ]:
cho mình code để tải file csv và tiền xử lý dữ liệu liên quan đến những file code chính này, nhớ đưa tất cả các hàm liên quan để kiểm tra với data nhỏ nha. với input là file mini_data.csv và output thi in ra file processed_data.csv nhá . cho mình bình luận bằng tiếng việt trong code nha và biểu diễn nó ngay tại màn hình này

In [ ]:
cho mình code tiếp theo sau code trên với giai đoạn chunking - tạo các hàm liên quan cho giai đoạn chunking (mới hoàn toàn, ko sd những file code sẵn này, nhớ viết tiếp sau khi code này và tạo lại các hàm liên quan thuộc về sematic chunk đã chạy nhá 